In [ ]:
import polars as pl

from utils import numeric_iso_col

In [3]:
kei = pl.read_csv('data/kei_6_years.csv')

In [4]:
num_kei = numeric_iso_col(kei, 'REF_AREA').drop_nulls('iso')

In [ ]:
kei_merge = num_kei['TIME_PERIOD', 'iso', 'OBS_VALUE'].rename(
    {'OBS_VALUE': 'Industrial Production Volume', 'TIME_PERIOD': 'year_month'}
    )

In [ ]:
acled = pl.read_csv('data/acled_oecd_countries_2018-2024.csv', schema_overrides={'iso': pl.String})

In [43]:
acled_year_month = acled.with_columns(
    pl.col('event_date')
    .str.to_date()
    .dt.strftime("%Y-%m")
    .alias('year_month'))
acled_grouped = acled_year_month.select(['year_month',
                                         'iso'])
acled_data = acled_grouped.group_by(['year_month',
                               'iso']).len().rename({'len':'incidents'})

In [45]:
data = acled_data.join(kei_merge, on=['year_month', 'iso'], how='left')

### Global Acute Food Insecurity
* https://data.humdata.org/dataset/global-acute-food-insecurity-country-data
* Only covers developing nations, not compatible with current OECD countries

In [47]:
ipc = pl.read_csv('data/ipc_global_national_long_latest_food_insecurity.csv')

### Global Mobility Report (Google):
* https://data.humdata.org/dataset/google-mobility-report
* Only covers 2020-2022

In [ ]:
global_mobility_report = pl.read_csv('data/Global_Mobility_Report.csv')